In [1]:
import sys
sys.path.append('..')
from osp import *

In [2]:
df = get_jstor_data()

In [3]:
df.decade.value_counts()

decade
2010    5346
2000    3736
1980    3481
1970    3249
1990    3136
1950    2634
1960    2629
2020    2018
1940    1901
1930    1726
1920    1147
1910     665
1900     609
1890     480
1880      26
Name: count, dtype: int64

In [4]:
def iter_jstor_sample():
    df = get_jstor_data().sample(frac=1)
    decs = list(df.groupby('decade'))
    maxn = max(len(decdf) for dec,decdf in decs)
    for i in range(maxn):
        for dec,decdf in decs:
            if len(decdf) > i:
                yield decdf.iloc[i]

In [5]:
row = next(iter_jstor_sample())

In [6]:
row

id                                    cca84e9b-d793-3a77-b2be-fc522adf12a6
review_required                                                       True
ithaka_doi                                                 10.2307/4543534
identifiers              {'print_isbn': None, 'online_isbn': None, 'pri...
title                                         The Philosophy of Revelation
is_part_of                         Proceedings of the Aristotelian Society
creators_string                                               J. Lightfoot
creators                 [{'first_name': 'J.', 'last_name': 'Lightfoot'...
publishers               [The Aristotelian Society, Oxford University P...
published_date                                                   1889-01-0
languages                                                            [eng]
discipline_names                                              [Philosophy]
issue_number                                                             2
issue_volume             

In [7]:
def parse_jstor_row(row, force=False):
    id = row['id']
    if not force and id in JSTOR_STASH:
        return stanza.Document.from_serialized(JSTOR_STASH[id])
    
    text = row['full_text']
    article = filter_pmla_pages(text)
    doc = get_nlp_doc(article)
    JSTOR_STASH[id] = doc.to_serialized()
    return doc

In [8]:
doc = parse_jstor_row(row)
tree_stats = pd.DataFrame(get_tree_stats(sent) for sent in doc.sentences)
tree_stats

,num_words,height,num_independent_clauses,num_dependent_clauses,num_words_in_dependent_clauses,num_words_in_independent_clauses,num_punct,num_punct_colon,num_punct_comma,num_parens,num_words_noun,num_words_verb,num_words_adjective,num_words_adverb
0,59,22,1,6,73,-14,7,2,2,0,11,9,3.0,2.0
1,51,14,1,3,28,23,4,0,3,0,9,5,6.0,3.0
2,19,11,1,1,9,10,1,0,0,0,4,2,4.0,2.0
3,25,10,0,1,25,0,4,0,3,0,5,4,1.0,NaN
4,12,7,1,0,0,12,2,0,1,0,3,3,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,15,11,1,1,6,9,1,0,0,0,4,2,NaN,1.0
118,35,21,1,1,25,10,2,0,1,0,7,4,4.0,3.0
119,21,13,2,0,0,21,3,0,0,2,4,2,2.0,1.0
120,30,19,1,3,27,3,2,0,1,0,3,7,2.0,2.0


In [9]:
tree_stats.mean()

num_words                           32.352459
height                              16.090164
num_independent_clauses              1.459016
num_dependent_clauses                1.631148
num_words_in_dependent_clauses      25.950820
num_words_in_independent_clauses     6.401639
num_punct                            2.811475
num_punct_colon                      0.040984
num_punct_comma                      1.557377
num_parens                           0.180328
num_words_noun                       6.532787
num_words_verb                       4.606557
num_words_adjective                  3.073394
num_words_adverb                     2.383838
dtype: float64

In [10]:
done = set(JSTOR_STASH.keys_l())
len(done)

6062

In [11]:
sample = [row for row in iter_jstor_sample() if row['id'] not in done]
len(sample)

26721

In [ ]:
for row in tqdm(sample):
    parse_jstor_row(row)

  7%|▋         | 1783/26721 [15:11:34<178:04:17, 25.71s/it]